# Code for training with validation
TODO: Implement early stopping with patience

In [3]:
import torch
import copy




def train_model(model, train_data, train_labels, val_data, val_labels, epochs, batch_size, loss_func, optimizer):
    
    print("Epoch\t train loss\t validation loss")
    best_model = copy.deepcopy(model.state_dict())
    best_loss = 10000 # bad value for dis, validation loss comparison

    for epoch in range(epochs):        
        epoch_train_loss = 0
        epoch_val_loss = 0
        model.train() # Set model to train mode
        
        for i in range(len(train_data)//batch_size): # BATCH SIZE MUST BE EVEN DIVIDER OF DATA LEN, otherwise we miss stuff here
            start = i*batch_size
            end = (i+1)*batch_size

            train_inputs = train_data[start:end]
            train_truth = train_labels[start:end]
            train_outputs = model(train_inputs)

            loss = loss_func(train_outputs, train_truth)
            
            #print("LOSS: ", loss)
            epoch_train_loss += loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        

        model.eval() # Set model to evaluation mode

        #val_inputs = torch.tensor(val_data, dtype = torch.float32, device = "cuda:0")
        #val_truth = torch.tensor(val_labels, dtype = torch.long, device = "cuda:0")
        val_outputs = model(val_data)
        val_loss = loss_func(val_outputs, val_labels)
        epoch_val_loss += val_loss

        # Check for new best model, this should be on val_data instead
        if epoch_val_loss < best_loss:
            best_model = copy.deepcopy(model.state_dict())

        print(epoch, "\t ", epoch_train_loss.item()/(len(train_data)//batch_size), "\t", epoch_val_loss.item())

    model.load_state_dict(best_model) # Set model to best performing one.
    


# TODO: Run on cuda.
# TODO: Add validation data and labels to train_model method params [DONE]
# TODO: Implement early stopping.
# TODO: Graphing of train and val loss, save train and val loss & perhaps accuracy to plot later.


def accuracy_check(network, data, labels):
    network.eval()

    # Accuracy check
    r = network(data)
    p = torch.max(r,1)[1]
    c = torch.sum(p == labels)
    print("ACCURACY:", c.item()/len(p))